In [ ]:
import pandas as pd
from support_codigo

def aguada(gps: pd.DataFrame, device: pd.DataFrame, latitude: float, longitude: float, area: float)->pd.DataFrame:
    '''
    Esta función retorna las coordenadas geográficas de la aguada en el lote dado.
    :param gps: Dataframe de gps para obtener la ubicación del potrero.
    :param device: Dataframe de gps para obtener los puntos fijos.
    :param latitude: Coordenada, latitud del potrero.
    :param longitude: Coordenada, longitud del potrero.
    :param area: Tamaño del potrero en hectareas.
    :return: Coordenadas (latitud y longitud) de la aguada.
    '''
    # Se encuentra el lote del potrero en el que se va a buscar la aguada.
    df_area_perimetro = filter_area_perimetro(gps, latitude, longitude, area)

    # Se buscan los puntos fijos en el dataframe de dispositivos.
    puntos_fijos = device[device.deviceType=='PUNTO FIJO']

    # Se eliminan las filas con dispositivos que no existen en el dataframe de gps.
    df_pf = puntos_fijos.drop(puntos_fijos[puntos_fijos['deviceMACAddress'].isin(['PRUEBA', 'MEVBE4FED3B7594'])].index)

    # Se unen los datasets por medio de las coincidencias en 'deviceMACAddress' y 'UUID'
    df_merged = pd.merge(df_pf, df_area_perimetro, left_on='deviceMACAddress', right_on='UUID')

    # Se eliminan las columnas que no son relevantes para esta función.
    df_merged = df_merged.drop(
        ['dataRowData_mac', 'dataRowData_battery', 'dataRowData_temperature', 'dataRowData_rssi',
         'dataRowData_accelerometer', 'deviceAnimalID', 'dataRowData_firmwareVersion',
         'dataRowData_voltage', 'dataRowData_charge', 'dataRowData_temperatureSoC', '__v_x', '__v_y',
         'deviceType','$addFields_dataRowData_GeoJSONUbication_type', 'deviceFirmwareVersion',
         '$addFields_dataRowData_GeoJSONUbication_coordinates','dataRowData_timestamp', 'payload',
         'geometry', 'dataRowData_gpsFixed','dataRowData_gpsVel', 'dataRowType'],
        axis=1
    )

    # Se crea una nueva columna con las coordenadas.
    df_merged['coordenadas'] = df_merged[
        ['dataRowData_lat', 'dataRowData_lng']
    ].apply(
        lambda x: ','.join(x.astype(str))
        , axis=1
    )

    # Se cuenta cuántas veces está cada valor en el dataframe final.
    df_agrupado = df_merged.groupby('coordenadas').agg({'UUID': 'count'}).reset_index()

    # Se agrupa por valores únicos en la columna 'UUID' y se crea una lista con las coordenadas.
    mask = df_agrupado['UUID'] == df_agrupado.UUID.unique().max()
    punto_fijo = df_agrupado.loc[mask, 'coordenadas'].tolist()

    # Se encuentra el UUID de las coordenadas del punto fijo.
    uuid = df_merged.groupby('coordenadas')['UUID'].count()
    max_uuid = uuid.max()
    most_common_uuid = df_merged[df_merged.groupby('coordenadas')['UUID'].transform('count') == max_uuid]['UUID'].iloc[0]

    # Separar las coordenadas en dos variables lat y lng.
    lat, lng = punto_fijo[0].split(',')
    lat = float(lat)
    lng = float(lng)

    # Definir un diccionario con los valores del UUID y las coordenadas.
    punto_fijo = {'UUID': [most_common_uuid],
                  'lat': [lat],
                  'lng': [lng]}

    # Convertir a dataframe la lista del punto fijo.
    df_punto_fijo = pd.DataFrame(punto_fijo)

    return df_punto_fijo